In [1]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.0.9000     ✔ purrr   0.3.4     
✔ tibble  3.0.1          ✔ dplyr   0.8.5     
✔ tidyr   1.0.3          ✔ stringr 1.4.0     
✔ readr   1.3.1          ✔ forcats 0.5.0     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



[1] "a6-essemble"

In [2]:
data <- read.csv("../input/a6-essemble/Earthquate_Damage.csv")

In [3]:
perf_eval_multi <- function(cm){
  
  # Simple Accuracy
  ACC = sum(diag(cm))/sum(cm)
  
  # Balanced Correction Rate
  BCR = 1
  for (i in 1:dim(cm)[1]){
    BCR = BCR*(cm[i,i]/sum(cm[i,])) 
  }
  
  BCR = BCR^(1/dim(cm)[1])
  
  return(c(ACC, BCR))
}

In [4]:
set.seed(42)
trn_idx <- sample(1:nrow(data), 150000)
train <- data[trn_idx,]
tmp <- data[-trn_idx,]
val_idx <- sample(1:nrow(tmp),50000)
val <- tmp[val_idx,]
test <- tmp[-val_idx,]
all_train <- rbind(train,val)

In [5]:
val$damage_grade <- as.factor(val$damage_grade)
train$damage_grade <- as.factor(train$damage_grade)
all_train$damage_grade <- as.factor(all_train$damage_grade)
test$damage_grade <- as.factor(test$damage_grade)

In [6]:
install.packages("adabag")
library(adabag)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: rpart

Loading required package: caret

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


The following object is masked from ‘package:httr’:

    progress


Loading required package: foreach


Attaching package: ‘foreach’


The following objects are masked from ‘package:purrr’:

    accumulate, when


Loading required package: doParallel

Loading required package: iterators

Loading required package: parallel



In [7]:
max_depth <- c(2,3,4)
mfinal <- c(10,20,30)

In [8]:
ada_pre_result = matrix(0,9,4)
colnames(ada_pre_result) <- c("mfinal","max_depth","ACC", "BCR")

In [9]:
iters_cnt <- 1
for(i in 1:3){
    for(j in 1:3){
        tmp_model <- boosting(damage_grade~., data=train[1:20000,], boos=TRUE, mfinal=mfinal[i],control=c(maxdepth=max_depth[j]))
        tmp_pred <- predict.bagging(tmp_model, newdata=val)
        ada_pre_result[iters_cnt,1] <- mfinal[i]
        ada_pre_result[iters_cnt,2] <- max_depth[j]
        ada_pre_result[iters_cnt,3:4] <- perf_eval_multi(tmp_pred$confusion)
        iters_cnt <- iters_cnt +1
    }
}

In [10]:
ada_pre_result

mfinal,max_depth,ACC,BCR
10,2,0.63460,0.6575046
10,3,0.62940,0.6833075
10,4,0.63324,0.6676789
20,2,0.57510,0.5089399
20,3,0.63494,0.6749944
20,4,0.62584,0.6668763
30,2,0.57546,0.5360569
30,3,0.63752,0.6545663
30,4,0.62846,0.6639726


In [12]:
model <- boosting(damage_grade~., data=all_train, boos=TRUE, mfinal=10,control=c(maxdepth=3))

In [13]:
pred <- predict.bagging(model, newdata=test)
perf_eval_multi(pred$confusion)

[1] 0.6338839 0.6667408